# OSS to SF Order Reconciliation

This notebook matches OSS billing orders to SF Orders using `OSS order_id` → `SF OSS_Order__c`.

## Purpose
- Identify which SF Orders correspond to actively billing OSS orders
- Filter migration scope to only billing orders
- Identify gaps between systems

## Key Matching
```
OSS.order_id = SF.Order.OSS_Order__c
```

---
**Created:** December 12, 2024

In [1]:
# === SETUP & IMPORTS ===

import sys
import pandas as pd
from simple_salesforce import Salesforce
from datetime import datetime

print(f"Python: {sys.executable}")
print(f"Pandas: {pd.__version__}")
print("✅ Imports successful")

Python: C:\Users\vjero\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe
Pandas: 2.2.3
✅ Imports successful


In [3]:
# === LOAD OSS DATA ===

# Load OSS billing data (already filtered for non-PA, active billing)
OSS_FILE = "active-non_pa-order-oss_12122025.csv"

oss_df = pd.read_csv(OSS_FILE, low_memory=False)
print(f"✅ Loaded OSS data: {len(oss_df):,} active non-PA orders")

# Create set of OSS order_ids for fast lookup
oss_order_ids = set(oss_df["order_id"].astype(str))
print(f"   Unique OSS order_ids: {len(oss_order_ids):,}")

✅ Loaded OSS data: 11,833 active non-PA orders
   Unique OSS order_ids: 11,833


In [5]:
# === CONFIGURATION ===

# ES (Source) Credentials
ES_USERNAME = "sfdcapi@everstream.net"
ES_PASSWORD = "pV4CAxns8DQtJsBq!"
ES_TOKEN = "r1uoYiusK19RbrflARydi86TA"
ES_DOMAIN = "login"  # or 'login' for production

# Output settings
TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")
OUTPUT_FILE = f"es_oss_sf_matched_{TIMESTAMP}.xlsx"

print("📋 Configuration loaded")

📋 Configuration loaded


In [6]:
# === CONNECT TO ES SALESFORCE ===

print("🔌 Connecting to ES Salesforce...")
es_sf = Salesforce(
    username=ES_USERNAME,
    password=ES_PASSWORD,
    security_token=ES_TOKEN,
    domain=ES_DOMAIN,
)
print(f"✅ Connected to ES: {es_sf.sf_instance}")

🔌 Connecting to ES Salesforce...
✅ Connected to ES: everstream.my.salesforce.com


In [7]:
# === QUERY SF ORDERS WITH OSS_Order__c ===

print("📊 Querying SF Orders with OSS_Order__c...")

# Query all Orders that have OSS_Order__c populated
sf_query = """
SELECT 
    Id, 
    Name,
    Service_ID__c,
    Status,
    AccountId,
    Account.Name,
    Billing_Invoice__c,
    Address_A__c,
    Address_Z__c,
    Node__c,
    OpportunityId,
    Service_Start_Date__c,
    Service_End_Date__c,
    Service_Provided__c,
    SOF_MRC__c,
    OSS_Service_ID__c,
    OSS_Order__c,
    Vendor_Circuit_ID__c,
    Primary_Product_Family__c,
    Primary_Product_Name__c,
    Dimension_4_Market__c
FROM Order
WHERE OSS_Order__c != null
ORDER BY OSS_Order__c
"""

try:
    result = es_sf.query_all(sf_query)
    sf_df = pd.DataFrame(result["records"])

    # Flatten Account.Name
    if "Account" in sf_df.columns:
        sf_df["Account_Name"] = sf_df["Account"].apply(
            lambda x: x.get("Name") if isinstance(x, dict) else None
        )
        sf_df = sf_df.drop(columns=["Account", "attributes"], errors="ignore")
    else:
        sf_df = sf_df.drop(columns=["attributes"], errors="ignore")

    print(f"\n✅ Retrieved {len(sf_df):,} SF Orders with OSS_Order__c")

except Exception as e:
    print(f"❌ Query error: {e}")
    raise

📊 Querying SF Orders with OSS_Order__c...

✅ Retrieved 42,479 SF Orders with OSS_Order__c


In [8]:
# === MATCH OSS TO SF ===

print("🔗 Matching OSS order_id to SF OSS_Order__c...")

# Convert SF OSS_Order__c to string for matching
sf_df["OSS_Order__c_str"] = sf_df["OSS_Order__c"].astype(str)

# Flag SF orders that match OSS billing
sf_df["In_OSS_Billing"] = sf_df["OSS_Order__c_str"].isin(oss_order_ids)

# Summary
matched = sf_df["In_OSS_Billing"].sum()
unmatched = len(sf_df) - matched

print(f"\n=== MATCHING RESULTS ===")
print(f"   SF Orders with OSS_Order__c: {len(sf_df):,}")
print(f"   ✅ Matched to OSS Billing: {matched:,}")
print(f"   ❌ NOT in OSS Billing: {unmatched:,}")

# Check reverse - OSS orders not in SF
sf_oss_ids = set(sf_df["OSS_Order__c_str"])
oss_not_in_sf = oss_order_ids - sf_oss_ids
print(f"\n   ⚠️ OSS orders NOT in SF: {len(oss_not_in_sf):,}")

🔗 Matching OSS order_id to SF OSS_Order__c...

=== MATCHING RESULTS ===
   SF Orders with OSS_Order__c: 42,479
   ✅ Matched to OSS Billing: 11,877
   ❌ NOT in OSS Billing: 30,602

   ⚠️ OSS orders NOT in SF: 9


In [9]:
# === ANALYZE MATCHED ORDERS ===

print("📊 Analyzing matched orders...")

matched_df = sf_df[sf_df["In_OSS_Billing"]].copy()

print(f"\n=== Matched Orders: {len(matched_df):,} ===")

# Status breakdown
print(f"\n--- By Status ---")
print(matched_df["Status"].value_counts().to_string())

# Market breakdown
print(f"\n--- By Market ---")
print(
    matched_df["Dimension_4_Market__c"].value_counts(dropna=False).head(15).to_string()
)

# BAN population
has_ban = matched_df["Billing_Invoice__c"].notna().sum()
no_ban = matched_df["Billing_Invoice__c"].isna().sum()
print(f"\n--- BAN Population ---")
print(f"   Has Billing_Invoice__c: {has_ban:,}")
print(f"   Missing Billing_Invoice__c: {no_ban:,}")

📊 Analyzing matched orders...

=== Matched Orders: 11,877 ===

--- By Status ---
Status
Activated                   11353
Disconnect in Progress        322
Disconnected                  113
Cancelled                      69
Deactivated                    14
Suspended (Late Payment)        5
Draft                           1

--- By Market ---
Dimension_4_Market__c
Cleveland                   4818
Michigan                    2372
Detroit                     1424
Indiana                     1113
Columbus                     799
Wisconsin                    771
Wisconsin Expansion          263
Missouri                     169
Illinois                      87
Kentucky                      56
Philadelphia                   3
Multi - Expansion Market       2

--- BAN Population ---
   Has Billing_Invoice__c: 11,872
   Missing Billing_Invoice__c: 5


In [10]:
# === ANALYZE UNMATCHED SF ORDERS ===

print("📊 Analyzing SF orders NOT in OSS billing...")

unmatched_df = sf_df[~sf_df["In_OSS_Billing"]].copy()

if len(unmatched_df) > 0:
    print(f"\n=== Unmatched SF Orders: {len(unmatched_df):,} ===")

    # Status breakdown
    print(f"\n--- By Status ---")
    print(unmatched_df["Status"].value_counts().to_string())

    # Market breakdown (might be PA)
    print(f"\n--- By Market ---")
    print(
        unmatched_df["Dimension_4_Market__c"]
        .value_counts(dropna=False)
        .head(15)
        .to_string()
    )

    # Check for PA markets
    PA_MARKETS = ["Pittsburgh", "Harrisburg", "Philadelphia", "Scranton", "Uniti-PA"]
    pa_count = unmatched_df["Dimension_4_Market__c"].isin(PA_MARKETS).sum()
    print(f"\n   PA Market orders in unmatched: {pa_count:,}")
else:
    print("✅ All SF orders with OSS_Order__c matched to OSS billing!")

📊 Analyzing SF orders NOT in OSS billing...

=== Unmatched SF Orders: 30,602 ===

--- By Status ---
Status
Disconnected              25875
Deactivated                2319
Cancelled                  1642
Activated                   743
Disconnect in Progress       23

--- By Market ---
Dimension_4_Market__c
Cleveland              10004
Michigan                8129
Detroit                 2471
Indiana                 2348
Wisconsin               1697
Columbus                1421
Uniti-PA                1404
Missouri                 774
Wisconsin Expansion      748
Illinois                 726
Philadelphia             525
Pittsburgh               159
Scranton                 127
Harrisburg                40
Kentucky                  29

   PA Market orders in unmatched: 2,255


In [11]:
# === IDENTIFY OSS ORDERS NOT IN SF ===

print("📊 Analyzing OSS orders NOT in SF...")

if len(oss_not_in_sf) > 0:
    # Get the OSS records not in SF
    oss_missing_df = oss_df[oss_df["order_id"].astype(str).isin(oss_not_in_sf)].copy()

    print(f"\n=== OSS Orders Not in SF: {len(oss_missing_df):,} ===")

    # Market breakdown
    print(f"\n--- By Market ---")
    print(oss_missing_df["market"].value_counts().to_string())

    # Order type breakdown
    print(f"\n--- By Order Type ---")
    print(oss_missing_df["order_type_cd"].value_counts().to_string())
else:
    print("✅ All OSS orders found in SF!")
    oss_missing_df = pd.DataFrame()

📊 Analyzing OSS orders NOT in SF...

=== OSS Orders Not in SF: 9 ===

--- By Market ---
market
Cleveland    3
Wisconsin    3
Michigan     2
Detroit      1

--- By Order Type ---
order_type_cd
NW    6
CH    3


In [12]:
# === CREATE MIGRATION-READY DATASET ===

print("📦 Creating migration-ready dataset...")

# Start with matched SF orders
migration_df = matched_df.copy()

# Merge with OSS data to get billing details
oss_for_merge = oss_df[
    [
        "order_id",
        "service_id",
        "account_id",
        "bill_start_date",
        "bill_end_date",
        "order_nm",
        "market",
        "expected_mrc",
    ]
].copy()
oss_for_merge["order_id"] = oss_for_merge["order_id"].astype(str)

migration_df = migration_df.merge(
    oss_for_merge,
    left_on="OSS_Order__c_str",
    right_on="order_id",
    how="left",
    suffixes=("", "_oss"),
)

print(f"\n✅ Migration dataset created: {len(migration_df):,} orders")

# Check data quality for migration
print(f"\n=== Data Quality for Migration ===")
print(f"   Has BAN: {migration_df['Billing_Invoice__c'].notna().sum():,}")
print(f"   Has A Location: {migration_df['Address_A__c'].notna().sum():,}")
print(f"   Has Z Location: {migration_df['Address_Z__c'].notna().sum():,}")
print(f"   Has Node: {migration_df['Node__c'].notna().sum():,}")

📦 Creating migration-ready dataset...

✅ Migration dataset created: 11,877 orders

=== Data Quality for Migration ===
   Has BAN: 11,872
   Has A Location: 11,815
   Has Z Location: 11,877
   Has Node: 1,700


In [14]:
# === EXPORT RESULTS ===

print(f"📁 Exporting to {OUTPUT_FILE}...")

with pd.ExcelWriter(OUTPUT_FILE, engine="openpyxl") as writer:

    # Summary
    summary = [
        {"Metric": "OSS Active Non-PA Orders", "Count": len(oss_df)},
        {"Metric": "SF Orders with OSS_Order__c", "Count": len(sf_df)},
        {"Metric": "✅ Matched (OSS ∩ SF)", "Count": len(matched_df)},
        {"Metric": "❌ SF not in OSS", "Count": len(unmatched_df)},
        {"Metric": "⚠️ OSS not in SF", "Count": len(oss_not_in_sf)},
        {"Metric": "", "Count": ""},
        {"Metric": "📦 MIGRATION SCOPE", "Count": len(migration_df)},
    ]
    pd.DataFrame(summary).to_excel(writer, sheet_name="Summary", index=False)

    # Migration-ready dataset
    migration_df.to_excel(writer, sheet_name="Migration_Ready", index=False)

    # SF orders not in OSS (for review)
    if len(unmatched_df) > 0:
        unmatched_df.to_excel(writer, sheet_name="SF_Not_In_OSS", index=False)

    # OSS orders not in SF (for review)
    if len(oss_missing_df) > 0:
        oss_missing_df.to_excel(writer, sheet_name="OSS_Not_In_SF", index=False)

print(f"\n✅ Export complete: {OUTPUT_FILE}")

📁 Exporting to es_oss_sf_matched_20251212_090432.xlsx...

✅ Export complete: es_oss_sf_matched_20251212_090432.xlsx


In [13]:
# === FINAL SUMMARY ===

print("=" * 70)
print("OSS TO SF RECONCILIATION - FINAL SUMMARY")
print("=" * 70)

print(
    f"""
📊 SOURCE COUNTS
   OSS Billing (Non-PA):        {len(oss_df):,}
   SF with OSS_Order__c:        {len(sf_df):,}

🔗 MATCHING RESULTS
   ✅ Matched (both systems):   {len(matched_df):,}
   ❌ SF not in OSS billing:    {len(unmatched_df):,}
   ⚠️ OSS not in SF:            {len(oss_not_in_sf):,}

📦 MIGRATION SCOPE
   Orders to migrate:           {len(migration_df):,}
   
   This represents orders that are:
   • Actively billing in OSS
   • Have matching SF Order record
   • Not in PA market

📁 OUTPUT: {OUTPUT_FILE}
"""
)

OSS TO SF RECONCILIATION - FINAL SUMMARY

📊 SOURCE COUNTS
   OSS Billing (Non-PA):        11,833
   SF with OSS_Order__c:        42,479

🔗 MATCHING RESULTS
   ✅ Matched (both systems):   11,877
   ❌ SF not in OSS billing:    30,602
   ⚠️ OSS not in SF:            9

📦 MIGRATION SCOPE
   Orders to migrate:           11,877
   
   This represents orders that are:
   • Actively billing in OSS
   • Have matching SF Order record
   • Not in PA market

📁 OUTPUT: es_oss_sf_matched_20251212_090432.xlsx

